load data from colab

In [3]:
!pip install -q kaggle

In [4]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"annanet","key":"e58d7795732c20cc54e293e03e998187"}'}

In [5]:
!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download 'm2m-mlm'

Dataset URL: https://www.kaggle.com/datasets/annanet/m2m-mlm
License(s): CC0-1.0
100% 4.97G/4.99G [01:10<00:00, 23.4MB/s]
100% 4.99G/4.99G [01:10<00:00, 75.9MB/s]


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import shutil

# Full path of
# the archive file
filename = "/content/m2m-mlm.zip"

# Target directory
extract_dir = "/content/drive/MyDrive/hackathon/m2m_mlm"

# Format of archive file
archive_format = "zip"

# Unpack the archive file
shutil.unpack_archive(filename, extract_dir, archive_format)
print("Archive file unpacked successfully.")

Archive file unpacked successfully.


In [13]:
!pip install sacrebleu
!pip install sqlalchemy
!pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import sacrebleu

In [15]:
df = pd.read_csv("/content/drive/MyDrive/hackathon/overall_80K.csv")

In [16]:
df

,Unnamed: 0,target,source
0,0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.
...,...,...,...
81141,2345,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄...,"Мне кажется, что подобные случаи могут вызыват..."
81142,2346,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.
81143,2347,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления мед..."
81144,2348,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэ...",Затем Уэйтс отправился на гастроли с такими му...


In [17]:
# чистим дф от мусора
df = df.drop(columns=[df.columns[0]])

In [18]:
df

,target,source
0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.
...,...,...
81141,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄...,"Мне кажется, что подобные случаи могут вызыват..."
81142,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.
81143,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления мед..."
81144,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэ...",Затем Уэйтс отправился на гастроли с такими му...


In [19]:
train_df, test_val_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

In [20]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [24]:
model_name = "facebook/m2m100_418M"
model_path = '/content/drive/MyDrive/hackathon/m2m_mlm'

model = M2M100ForConditionalGeneration.from_pretrained(model_path)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
tokenizer.add_special_tokens({'additional_special_tokens': ['<mns_MNS>']})
model.resize_token_embeddings(len(tokenizer))

M2M100ScaledWordEmbedding(128105, 1024, padding_idx=1)

In [26]:
def preprocess_function(examples):
    inputs = tokenizer(examples['source'], truncation=True, padding='max_length', max_length=128)
    targets = tokenizer(examples['target'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

In [27]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/64916 [00:00<?, ? examples/s]

Map:   0%|          | 0/8115 [00:00<?, ? examples/s]

Map:   0%|          | 0/8115 [00:00<?, ? examples/s]

In [28]:
train_dataset = train_dataset.remove_columns(['source', 'target', '__index_level_0__'])
val_dataset = val_dataset.remove_columns(['source', 'target', '__index_level_0__'])
test_dataset = test_dataset.remove_columns(['source', 'target', '__index_level_0__'])

In [29]:
import torch
torch.cuda.empty_cache()

In [30]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/hackathon/m2m+mlm_finetuned",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_total_limit=5,
    save_strategy="no",
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    report_to="none",
    fp16=True,
    load_best_model_at_end=False,
)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])
    chrf = sacrebleu.corpus_chrf(decoded_preds, [decoded_labels])

    return {"bleu": bleu.score, "chrf": chrf.score}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


KeyboardInterrupt: 

In [32]:
model.eval();

In [ ]:
model.save_pretrained("/content/drive/MyDrive/hackathon/m2m+mlm_finetuned/model/")
tokenizer.save_pretrained("/content/drive/MyDrive/hackathon/m2m+mlm_finetuned/tokenizer/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


In [ ]:
import random
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

random.seed(42)
random_test_samples = test_dataset

def generate_translation(sample):
    input_ids = torch.tensor(sample['input_ids']).unsqueeze(0).to(device)  # Конвертируем список в тензор
    with torch.no_grad():
        generated_ids = model.generate(input_ids, max_length=200, num_beams=5, early_stopping=True)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

original_texts = []
correct_translations = []
model_translations = []

for sample in tqdm(random_test_samples):
    input_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
    correct_translation = tokenizer.decode(sample['labels'], skip_special_tokens=True)
    model_translation = generate_translation(sample)

    original_texts.append(input_text)
    correct_translations.append(correct_translation)
    model_translations.append(model_translation)

df_results = pd.DataFrame({
    "Original Text": original_texts,
    "Correct Translation": correct_translations,
    "Model Translation": model_translations
})

df_results.loc[:, 'Original Text'] = df_results['Original Text'].str.replace('__en__', '')
df_results.loc[:, 'Correct Translation'] = df_results['Correct Translation'].str.replace('__en__', '')
df_results.loc[:, 'Model Translation'] = df_results['Model Translation'].str.replace('__en__', '')

bleu_score = sacrebleu.corpus_bleu(df_results['Model Translation'].tolist(),
                                   [df_results['Correct Translation'].tolist()]).score
chrf_score = sacrebleu.corpus_chrf(df_results['Model Translation'].tolist(),
                                   [df_results['Correct Translation'].tolist()]).score

print(f"BLEU Score: {bleu_score}")
print(f"ChrF Score: {chrf_score}")

df_results.head(50)


100%|██████████| 8115/8115 [1:12:39<00:00,  1.86it/s]


BLEU Score: 21.664822458079087
ChrF Score: 51.97204285133187


,Original Text,Correct Translation,Model Translation
0,Пётр всего полведра воды принёс.,Петыр вēтра суп вит туп та тотас.,Пēтыр пуссын во̄т па̄л вит тотыс.
1,Он как я ростом стал.,Тав ам янытумый ēмтыс,Тав ам хумле яныгмаыг ēмтыс.
2,"Стоят лодки полные рыбы, для заготовки запасо...","Хл ха̄пыт нлэ̄гыт, та̄юм ва̄рнэ та̄юм савакыт...","Хл та̄глы ха̄пыт нлэ̄гыт, тэ̄нутыт щё̄питан м..."
3,Глаза имеющий человек его может увидеть.,Сам ōньщнэ хōтпан тав кāсалалаве.,Сама хум таве ка̄салакве вēрми.
4,"А я говорю: "" Наплевать, что больно, Маруся.","А а̄м ла̄вегум: "" Са̄лямак, ма̄ныр а̄ймы, Мар...","Ам ла̄ве̄гум: « Маруся, маныр а̄гмы."
5,"Пришли к купцу, домой зашли.","Щёлы хум палт ёхтысый., юв щялтсы̄й.","Ва̄тахум палт ёхтысы̄г, юв щалтсы̄г."
6,"Ты говоришь, как взрослые!",На яныгма̄хум хольт потыртэ̄гын!,"На ла̄вēгын, яныгхо̄тпат хольт!"
7,Он мы манси и ханты работающие в газете люди ...,Тав ма̄н ма̄ньщи ос ханты газетагувт рпитан м...,Тав ма̄н ма̄ньщи ос ханты газетат рпитан ма̄х...
8,"Затем человек, одетый в светлый халат, в свет...","Тувыл ань войкан сахи, войкан ня̄ра масум хум...","Тувыл посы супыл масхатым, посы ла̄гыл масхат..."
9,Ползающие звери - семь юров пусть ползают.,Ва̄кынэ й са̄т ю̄р вос ва̄кēгыт.,Во̄р й са̄т ю̄р вос ргалаве.
